# Agenda

1. What are decorators?
2. Decorating functions
3. Outer functions and decorators
4. Using the outer function in the decorator
5. Arguments to decorators
6. Decorating classes
7. Writing decorators as classes
8. Fixing some tiny decorator issues

In [2]:
def a():
    return 'A!'

def b():
    return 'B!'

print(a())
print(b())

A!
B!


In [3]:
# put lines above and below each printout

lines = '-' * 60 + '\n'

def a():
    return f'{lines}A!\n{lines}'

def b():
    return f'{lines}B!\n{lines}'

print(a())
print(b())

------------------------------------------------------------
A!
------------------------------------------------------------

------------------------------------------------------------
B!
------------------------------------------------------------



# About functions

1. Functions are objects, just like everything else in Python.
2. When I use `def`, I'm doing two things:
    - Creating a function object
    - Assigning that function object to a variable
3. When I assign a varible in a function, then that variable is local
4. I can pass functions as arguments to other functions
5. A function can return any type of object, including another function

In [5]:
# use a function

lines = '-' * 60 + '\n'

def with_lines(func):
    return f'{lines}{func()}{lines}'

def a():
    return f'A!\n'

def b():
    return f'B!\n'

print(with_lines(a))
print(with_lines(b))

------------------------------------------------------------
A!
------------------------------------------------------------

------------------------------------------------------------
B!
------------------------------------------------------------



In [6]:
# use a function that returns a function

lines = '-' * 60 + '\n'

def with_lines(func):
    def wrapper():
        return f'{lines}{func()}{lines}'
    return wrapper

def a():
    return f'A!\n'
with_lines_a = with_lines(a)


def b():
    return f'B!\n'
with_lines_b = with_lines(b)

print(with_lines_a())
print(with_lines_b())

------------------------------------------------------------
A!
------------------------------------------------------------

------------------------------------------------------------
B!
------------------------------------------------------------



In [7]:
# assign the result of calling with_lines back to the original function's name

lines = '-' * 60 + '\n'

def with_lines(func):  # closure
    def wrapper():
        return f'{lines}{func()}{lines}'
    return wrapper

def a():
    return f'A!\n'
a = with_lines(a)


def b():
    return f'B!\n'
b = with_lines(b)

print(a())
print(b())

------------------------------------------------------------
A!
------------------------------------------------------------

------------------------------------------------------------
B!
------------------------------------------------------------



In [8]:
# now, let's make it into a real decorator

lines = '-' * 60 + '\n'

def with_lines(func):  # outer function is called once, just after the decorated function is defined

    def wrapper():     # inner function is called INSTEAD OF the decorated function
        return f'{lines}{func()}{lines}'
    return wrapper

@with_lines
def a():
    return f'A!\n'
# a = with_lines(a)


@with_lines
def b():
    return f'B!\n'
# b = with_lines(b)

print(a())
print(b())

------------------------------------------------------------
A!
------------------------------------------------------------

------------------------------------------------------------
B!
------------------------------------------------------------



In [9]:
a.__name__

'wrapper'

In [10]:
b.__name__

'wrapper'

In [14]:
# what about arguments?

lines = '-' * 60 + '\n'

def with_lines(func):    # outer function is called once, just after the decorated function is defined

    def wrapper(*args):  # inner function is called INSTEAD OF the decorated function
        return f'{lines}{func(*args)}{lines}'  # unrolling of args in our call to func
    return wrapper

@with_lines
def a():
    return f'A!\n'
# a = with_lines(a)

@with_lines
def b():
    return f'B!\n'
# b = with_lines(b)

@with_lines
def add(first, second):
    return f'{first + second}\n'

print(a())
print(b())
print(add(3, 5))    # passing 2 positional arguments, because add requires two positional arguments

------------------------------------------------------------
A!
------------------------------------------------------------

------------------------------------------------------------
B!
------------------------------------------------------------

------------------------------------------------------------
8
------------------------------------------------------------



# To write a decorator

1. Write an outer function that has one parameter, `func`, which will be assigned the decorated function.
2. The body of the outer function will define a new function, traditionally called `wrapper`.
3. `wrapper` should take `*args` as an argument.
4. The return value from `wrapper` will replace the return value from the original (decorated) function.  It can include the original function's output, but it doesn't have to.
5. The outer function returns `wrapper`, which is assigned to the decorated function's name.

# So what?

- Redirect `stdout` or `stderr` for logging to a file
- Check permissions before running a function
- Check the date/time before running a function
- Check the timing of a function's execution, and log it
- Transform arguments before they're ever handed to the function
- Transform outputs before they're returned by the function

# Exercise: Timing decorator

1. Write two functions, `add` and `mul`.  These should add and multiply numbers, respectively.  Add a `time.sleep` in them, so that things won't run immediately.  You can say something like `time.sleep(random.randint(0, 3))`.
2. Write a decorator, `timefunc`, which will run a decorated function, and will return the original value.  So the decorator won't change the inputs or outputs.
3. The decorator will, however, keep track of how long the inner function was running.  You can use `time.perf_counter` to get the time in seconds, both before and after.
4. Before returning its value, have the decorator write to a logfile the current time (`time.time`), the function's name (`__name__`), and the time it took to run.

Example:

    print(add(2,3))
    print(mul(3, 6))
    
After that, we should see something like this in our logfile, `timelog.txt`:

    123 add 3
    125 mul 2
    